In [6]:
import os
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"  # GPU 0
os.environ["TF_FORCE_GPU_ALLOW_GROWTH"] = "true"
from deepface import DeepFace
import pandas as pd
import tensorflow as tf

print("TensorFlow version:", tf.__version__)
print("GPU détecté:", tf.config.list_physical_devices('GPU'))

TensorFlow version: 2.20.0
GPU détecté: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [9]:
dfs = DeepFace.find(img_path=r"../public/albums/mic_2025_amiens/100_2037.JPG",
                    db_path=r"../public/albums/mic_2025_amiens/",
                    enforce_detection=False,
                    model_name="ArcFace",
                    )
print(dfs)

25-10-21 12:03:47 - Searching ../public/albums/mic_2025_amiens/100_2037.JPG in 282 length datastore
25-10-21 12:03:48 - find function duration 3.336427688598633 seconds
[                                             identity  \
0       ../public/albums/mic_2025_amiens/100_2037.JPG   
1       ../public/albums/mic_2025_amiens/100_2008.JPG   
2   ../public/albums/mic_2025_amiens/test/100_2008...   
3       ../public/albums/mic_2025_amiens/100_2045.JPG   
4       ../public/albums/mic_2025_amiens/100_2095.JPG   
5       ../public/albums/mic_2025_amiens/100_2129.JPG   
6       ../public/albums/mic_2025_amiens/100_2092.JPG   
7       ../public/albums/mic_2025_amiens/100_2038.JPG   
8       ../public/albums/mic_2025_amiens/100_2093.JPG   
9       ../public/albums/mic_2025_amiens/100_2094.JPG   
10      ../public/albums/mic_2025_amiens/100_2140.JPG   
11      ../public/albums/mic_2025_amiens/100_2035.JPG   

                                        hash  target_x  target_y  target_w  \
0   da5524

In [10]:
import cv2
import numpy as np

df = dfs[0]

for i, row in df.iterrows():
    img_path = row["identity"]
    # print("Chemin absolu :", os.path.abspath(img_path))
    img = cv2.imread(img_path)

    if img is None:
        print("⚠️ Impossible de lire :", img_path)
        continue

    # Récupération et validation des coordonnées
    x, y, w, h = int(row["target_x"]), int(row["target_y"]), int(row["target_w"]), int(row["target_h"])

    # Sécurité : s'assure que le rectangle ne dépasse pas l'image
    h_img, w_img = img.shape[:2]
    x = max(0, min(x, w_img - 1))
    y = max(0, min(y, h_img - 1))
    w = max(1, min(w, w_img - x))
    h = max(1, min(h, h_img - y))

    # ✅ Rectangle noir
    cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 4)

    # Label de confiance
    label = f"{row['confidence']:.1f}%"
    cv2.putText(img, label, (x, max(30, y - 10)), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 3)

    cv2.imshow("Visage", cv2.resize(np.array(img, dtype=np.uint8), None, fx=.25, fy=.25))

    key = cv2.waitKey(0)
    if key == 27:  # Échap pour quitter
        break

cv2.destroyAllWindows()